In [16]:
from ludwig.datasets import agnews
import logging
import pandas as pd

# Loads the dataset as a pandas.DataFrame
train_df, test_df, _ = agnews.load(split=True)


In [17]:
config = {
  "input_features": [
    {
      "name": "title",            # The name of the input column
      "type": "text",             # Data type of the input column
      "encoder": {
            "type": "parallel_cnn"
       }                          # The model architecture we should use for encoding this column
    }
  ],
  "output_features": [
    {
      "name": "class",
      "type": "category",
    }
  ],
  "trainer": {
    "epochs": 3,  # We'll train for three epochs. Training longer might give
                  # better performance.
  }
}

In [18]:
from ludwig.api import LudwigModel

# Constructs Ludwig model from config dictionary
model = LudwigModel(config, logging_level=logging.INFO)

# Trains the model. This cell might take a few minutes.
train_stats, preprocessed_data, output_directory = model.train(dataset=train_df)


╒════════════════════════╕
│ EXPERIMENT DESCRIPTION │
╘════════════════════════╛

╒══════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                                           │
├──────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                                      │
├──────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /Users/shuai/PycharmProjects/automl/results/api_experiment_run                                           │
├──────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ludwig_version   │ 

/opt/homebrew/anaconda3/envs/neural_style_transfer/lib/python3.8/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1004.)
  return F.conv1d(input, weight, bias, self.stride,


Throughput at batch_size=2: 131.94407 samples/s
Exploring batch_size=4
Throughput at batch_size=4: 228.71888 samples/s
Exploring batch_size=8
Throughput at batch_size=8: 403.08045 samples/s
Exploring batch_size=16
Throughput at batch_size=16: 159.35011 samples/s
Throughput decrease at batch_size=16
Selected batch_size=8

╒══════════╕
│ TRAINING │
╘══════════╛

Creating fresh model training run.
Training for 30162 step(s), approximately 3 epoch(s).
Early stopping policy: 5 round(s) of evaluation, or 50270 step(s), approximately 5 epoch(s).

Starting with step 0, epoch: 0
Training:  33%|███▎      | 10053/30162 [03:45<08:44, 38.31it/s]
Running evaluation for step: 10054, epoch: 0
Evaluation test : 100%|██████████| 2873/2873 [00:13<00:00, 212.59it/s]
Evaluation took 19.5422s

╒════════════╤════════════╤═════════════╤════════╤═══════════╕
│ class      │   accuracy │   hits_at_k │   loss │   roc_auc │
╞════════════╪════════════╪═════════════╪════════╪═══════════╡
│ train      │     0.8245 │ 

In [19]:
# Generates predictions and performance statistics for the test set.
test_stats, predictions, output_directory = model.evaluate(
  test_df,
  collect_predictions=True,
  collect_overall_stats=True
)

Evaluation: 100%|██████████| 950/950 [00:03<00:00, 248.92it/s]

===== class =====
accuracy: 0.8651036024093628
hits_at_k: 0.9880257248878479
loss: 0.46574562788009644
overall_stats: { 'avg_f1_score_macro': 0.864559968959744,
  'avg_f1_score_micro': 0.8651138307672062,
  'avg_f1_score_weighted': 0.8645654962014254,
  'avg_precision_macro': 0.8652806336338054,
  'avg_precision_micro': 0.8651138307672062,
  'avg_precision_weighted': 0.8651138307672062,
  'avg_recall_macro': 0.8651035863751004,
  'avg_recall_micro': 0.8651138307672062,
  'avg_recall_weighted': 0.8651138307672062,
  'kappa_score': 0.8201510943468129,
  'token_accuracy': 0.8651138307672062}
per_class_stats: {sci_tech: {   'accuracy': 0.9210422424003158,
    'f1_score': 0.846311475409836,
    'fall_out': 0.06176522196876644,
    'false_discovery_rate': 0.17564870259481036,
    'false_negative_rate': 0.1305263157894737,
    'false_negatives': 248,
    'false_omission_rate': 0.04432529043789102,
    'false_positive_rate': 0.061

In [21]:
from ludwig.visualize import confusion_matrix

confusion_matrix(
  [test_stats],
  model.training_set_metadata,
  'class',
  top_n_classes=[5],
  model_names=[''],
  normalize=True,
)

/opt/homebrew/anaconda3/envs/neural_style_transfer/lib/python3.8/site-packages/ludwig/utils/visualization_utils.py:1167: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([""] + labels, rotation=45, ha="left")
/opt/homebrew/anaconda3/envs/neural_style_transfer/lib/python3.8/site-packages/ludwig/utils/visualization_utils.py:1168: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([""] + labels, rotation=45, ha="right")


In [22]:
# Visualizes learning curves, which show how performance metrics changed over
# time during training.
from ludwig.visualize import learning_curves

learning_curves(train_stats, output_feature_name='class')

In [23]:
text_to_predict = pd.DataFrame({
  "title": [
    "Google may spur cloud cybersecurity M&A with $5.4B Mandiant buy",
    "Europe struggles to meet mounting needs of Ukraine's fleeing millions",
    "How the pandemic housing market spurred buyer's remorse across America",
  ]
})

predictions, output_directory = model.predict(text_to_predict)

Prediction: 100%|██████████| 1/1 [00:00<00:00, 293.53it/s]


In [24]:
predictions

,class_predictions,class_probabilities,class_probability,class_probabilities_sci_tech,class_probabilities_world,class_probabilities_sports,class_probabilities_business
0,sci_tech,"[0.9999967813491821, 7.663726875328791e-10, 1....",0.999997,0.999997,7.663727e-10,1.123789e-10,0.000003
1,world,"[0.05853056535124779, 0.9117871522903442, 0.00...",0.911787,0.058531,9.117872e-01,2.261778e-03,0.027420
2,business,"[0.3341456651687622, 0.08625838905572891, 0.00...",0.573059,0.334146,8.625839e-02,6.536994e-03,0.573059
